## Pre-trained image recognition networks
Uses pre-trained ConvNet networks trained for the ImageNet challenge.

In [2]:
# Import the necessary packages
from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from IPython.display import Image
import numpy as np

# Ignore deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Available models in Keras
MODELS = {
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception, # TensorFlow ONLY
    "resnet": ResNet50
}

# Loads the pre-trained model
def load_model(model):
    # Check that the model is valid
    if model not in MODELS.keys():
        raise AssertionError("Invalid model {}".format(model))

    # Initialize the input image shape (224x224 pixels) along with
    # the pre-processing function (this might need to be changed
    # based on which model we use to classify our image)
    inputShape = (224, 224)
    preprocess = imagenet_utils.preprocess_input

    # If we are using the InceptionV3 or Xception networks, then we
    # need to set the input shape to (299x299) [rather than (224x224)]
    # and use a different image processing function
    if model in ("inception", "xception"):
        inputShape = (299, 299)
        preprocess = preprocess_input

    # Load our the network weights
    print("Loading model {}".format(model))
    Network = MODELS[model]
    model = Network(weights="imagenet")
    return model,inputShape,preprocess

# Classifies an image, i.e. tries to find what's in the picture
def classify_image(image_path):
    # Load the input image using the Keras helper utility while ensuring
    # the image is resized to `inputShape`, the required input dimensions
    # for the ImageNet pre-trained network
    print("Loading and pre-processing image '{}'".format(image_path))
    image = load_img(image_path, target_size=inputShape)
    image = img_to_array(image)

    # Our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through the network
    image = np.expand_dims(image, axis=0)

    # Pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess(image)

    # Classify the image
    print("Classifying image with model '{}'".format(model.name))
    preds = model.predict(image)
    P = imagenet_utils.decode_predictions(preds)

    # Loop over the predictions and display the rank-5 predictions +
    # probabilities to our terminal
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("{}. {}: {:.2f}%".format(i + 1, label, prob * 100))

# Load the model we want
model,inputShape,preprocess = load_model("xception")

Loading model xception


In [ ]:
img = "images/soccer_ball.jpg"
classify_image(img)
Image(img, width=500)

In [ ]:
img = "images/boat.png"
classify_image(img)
Image(img, width=500)